# More Practice With SQL Queries - Lab

## Introduction

In this lesson, we'll run through some practice questions to refresh your knowledge of SQL Queries!

## Objectives

You will be able to:

- Use `GROUP BY` statements in SQL to apply aggregate functions like: `COUNT`, `MAX`, `MIN`, and `SUM`
- Decide and perform whichever type of join is best for retrieving desired data
- Use the `HAVING` clause to compare different aggregates
- Write subqueries to decompose complex queries

## Getting Started

As in previous labs, we'll make use of the `sqlite3` library as well as Pandas. By combining them, we'll be able to write queries as Python strings, and make sure that the results are always returned as a Pandas DataFrame. 

We'll start by loading both libraries and connecting to the database we'll be using for this lab, `data.sqlite`. You may remember this database from a previous lab. As a refresher, here's the ERD diagram for this database: 

<img src='images/Database-Schema.png'>

In the cell below:

* Import the necessary libraries `pandas` and `sqlite3` 
* Establish a connection to the database `data.sqlite` 
* Get the `cursor` from the connection and store it in the variable `c` 

In [2]:
import pandas as pd 
import sqlite3
conn = sqlite3.connect('data.sqlite')
c = conn.cursor()

## Basic Queries

Now, let's review basic SQL queries. In the cell below:

Write a query that gets the first name, last name, phone number, address, and credit limit for all customers in California with a credit limit greater than 25000.00. 

In [8]:
# For the first query, the boilerplate for getting 
# the query into a DataFrame has been provided for you
c.execute("""select contactFirstName, contactLastName, phone, addressLine1, addressLine2, creditLimit
          from customers 
          where creditLimit > 25000.00""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,contactFirstName,contactLastName,phone,addressLine1,addressLine2,creditLimit
0,Jean,King,7025551838,8489 Strong St.,,71800
1,Peter,Ferguson,03 9520 4555,636 St Kilda Road,Level 3,117300
2,Janine,Labrune,40.67.8555,"67, rue des Cinquante Otages",,118200
3,Jonas,Bergulfsen,07-98 9555,Erling Skakkes gate 78,,81700
4,Susan,Nelson,4155551450,5677 Strong St.,,210500
...,...,...,...,...,...,...
89,Rosa,Salazar,2155559857,11328 Douglas Av.,,72600
90,Sue,Taylor,4155554312,2793 Furth Circle,,60300
91,Thomas,Smith,(171) 555-7555,120 Hanover Sq.,,43300
92,Valarie,Franco,6175552555,6251 Ingle Ln.,,85100


#### Expected Output

<img src='images/expected-output-1.png'>

## Aggregate Functions and GROUP BY

Next, write a query that gets the average credit limit per state.

In [11]:
c.execute("""select state, AVG(creditLimit)
          from customers 
          group by state
          """)
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,state,AVG(creditLimit)
0,,61839.726027
1,BC,89950.000000
2,CA,83854.545455
3,CT,57350.000000
4,Co. Cork,0.000000
5,Isle of Wight,93900.000000
6,MA,70755.555556
7,NH,114200.000000
8,NJ,43000.000000
9,NSW,100550.000000


#### Expected Output

<img src='images/expected-output-2.png'>

## JOINs

Now, write a query that uses JOIN statements to get the customer name, customer number, order number, status, and quantity ordered. Print only the head of this DataFrame. 

In [16]:
c.execute("""select customerName, customerNumber, orderNumber, status, quantityOrdered
          from customers 
          join orders
          using(customerNumber)
          join orderdetails
          using(orderNumber);""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,customerName,customerNumber,orderNumber,status,quantityOrdered
0,Atelier graphique,103,10123,Shipped,26
1,Atelier graphique,103,10123,Shipped,34
2,Atelier graphique,103,10123,Shipped,46
3,Atelier graphique,103,10123,Shipped,50
4,Atelier graphique,103,10298,Shipped,32
...,...,...,...,...,...
2991,Kelly's Gift Shop,496,10399,Shipped,32
2992,Kelly's Gift Shop,496,10399,Shipped,40
2993,Kelly's Gift Shop,496,10399,Shipped,51
2994,Kelly's Gift Shop,496,10399,Shipped,57


#### Expected Output

<img src='images/joins.png' width='500' height='600'> 

## HAVING and ORDER BY

Now, return the customerName, customerNumber, productName, productCode, and total number ordered for any product a customer has bought 10 or more of cumulatively. Sort the rows in descending order by the quantity ordered. 

**_Hint_**: For this one, you'll need to make use of HAVING, GROUP BY, and ORDER BY -- make sure you get the order of them correct!

In [45]:
c.execute("""select customerName, customerNumber, productName, productCode, count(quantityOrdered) as TotalOrdered
            from customers
            join orders
            using(customerNumber)
            join orderdetails
            using(orderNumber)
            join products
            using(productCode)
            group by customerName
            having quantityOrdered >= 10
            order by totalOrdered desc""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,customerName,customerNumber,productName,productCode,TotalOrdered
0,Euro+ Shopping Channel,141,1969 Corvair Monza,S12_3148,259
1,Mini Gifts Distributors Ltd.,124,1958 Setra Bus,S12_1666,180
2,"Australian Collectors, Co.",114,1996 Moto Guzzi 1100i,S10_2016,55
3,La Rochelle Gifts,119,1969 Harley Davidson Ultimate Chopper,S10_1678,53
4,"AV Stores, Co.",187,1965 Aston Martin DB5,S18_1589,51
...,...,...,...,...,...
93,Auto-Moto Classics Inc.,198,1999 Yamaha Speed Boat,S18_3029,8
94,Frau da Collezione,473,P-51-D Mustang,S18_2581,8
95,Royale Belge,381,1950's Chicago Surface Lines Streetcar,S32_3207,8
96,Atelier graphique,103,1965 Aston Martin DB5,S18_1589,7


#### Expected Output

<img src='images/having_order.png'>

## Subqueries

Finally, get the first name, last name, employee number, and office code for employees from offices with less than 5 employees. Print the first five rows of this DataFrame. 

In [47]:
c.execute("""select firstName, lastName, employeeNumber, officeCode 
            from employees
            where officeCode in (select officeCode from offices
            join employees
            using(officeCode)
            group by officeCode
            having count(employeeNumber) < 5);""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,firstName,lastName,employeeNumber,officeCode
0,William,Patterson,1088,6
1,Julie,Firrelli,1188,2
2,Steve,Patterson,1216,2
3,Foon Yue,Tseng,1286,3
4,George,Vanauf,1323,3
5,Larry,Bott,1501,7
6,Barry,Jones,1504,7
7,Andy,Fixter,1611,6
8,Peter,Marsh,1612,6
9,Tom,King,1619,6


#### Expected Output

<img src='images/expected-output-5.png'>

# Summary

In this lesson, we reviewed all the major concepts and keywords associated with SQL queries!